# Sentiment analysis 



Documentation: https://textblob.readthedocs.io/en/

Preparation et chargement du corpus

In [1]:
from pathlib import Path
import re


data_dir = Path("../../data/txt")

all_txt = list(data_dir.glob("*.txt"))

YEAR = "1950"
pat = re.compile(rf"{YEAR}")

files_year = [p for p in all_txt if pat.search(p.name)]
print(f"Fichiers trouvés pour {YEAR}: {len(files_year)}")

corpus_year = ""
for path in files_year:
    for enc in ("utf-8", "latin-1", "cp1252"):
        try:
            with open(path, "r", encoding=enc, errors="ignore") as f:
                corpus_year += f.read() + "\n"
            break
        except Exception:
            continue

print(f"Taille du corpus {YEAR}: {len(corpus_year):,} caractères")
print("\nExtrait:\n", corpus_year[:500])

Fichiers trouvés pour 1950: 4
Taille du corpus 1950: 133,895 caractères

Extrait:
 L'AVENIR DU LUXEMBOURG Samedi 15 avri j 350, 
MORHET 
Soirée dramatique 
1 Le cercle dramatique Sainte-Cécile 
de Morhet reprendra, ce dimanche 16 
avril ^Quasimodo), sa brillante soirée 
qui a remporté un succès si remarqua-
| bie le 10 mars dernier. 
i Rappelons ie programme : 
; 1) ouverture : « Brabançonne »,par 
• la Fantare ; 2) « La .bohème », chœur 
à 2 voix exécuté par JV^.es Renée Cara, 
j Josée Goffin, Anyse Hubermont et Hé-
f lène Bellanger ; a) La comédie en deux 
actes de Marcell* 


In [2]:
!python -m pip install -q textblob textblob-fr


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Import

In [6]:
import pandas as pd
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

Decoupe du corpus en phrases

In [7]:
sentences = re.split(r'(?<=[\.\?\!])\s+', corpus_year)
len(sentences), sentences[:3]

(2583,
 ["L'AVENIR DU LUXEMBOURG Samedi 15 avri j 350, \nMORHET \nSoirée dramatique \n1 Le cercle dramatique Sainte-Cécile \nde Morhet reprendra, ce dimanche 16 \navril ^Quasimodo), sa brillante soirée \nqui a remporté un succès si remarqua-\n| bie le 10 mars dernier.",
  "i Rappelons ie programme : \n; 1) ouverture : « Brabançonne »,par \n• la Fantare ; 2) « La .bohème », chœur \nà 2 voix exécuté par JV^.es Renée Cara, \nj Josée Goffin, Anyse Hubermont et Hé-\nf lène Bellanger ; a) La comédie en deux \nactes de Marcell* Joignet « L'escalier \n' du Monastère de -viarmoutier », qui se-\n; ra interDretée par Josée Goffin, Marie-\nLouise Flamant,, Madeleine Ska, Renée \nCara, Solange Noirot, Jeanine Grand-\n!",
  'iean,.'])

Filtrage et choix de 10 phrases

In [5]:
clean_sentences = [s.strip() for s in sentences 
                   if len(s.split()) >= 6 and not re.search(r"[@^~%$§<>]", s)]

# Prendre les 10 premières
sample_sentences = clean_sentences[:10]

for i, s in enumerate(sample_sentences, 1):
    print(f"{i}. {s}\n")

1. Madeleine Van Mullem ; 4) « La 
Vierge du Portail de Reims », 1 acte en 
prose de Philippe Gérard, interprété 
par Renée Lambert, Madeleine Ska, 
j Renée Cara, Jeanine Grandjean, Ma-
j rie-Louise Flamant, Marguerite Maquet 
', et Solange Grandjean ; 5) La comédie 
,' en 2 actes de M.

2. Cartier-Bresson « L'E-
: toile », Interorétée par Madeleine Ska, 
' Jeanine Grandjean, Marguerite Ma-
3uet, Jeanne Bays, Solange Noirot, Ma-
eleine Van Mullem, Marie-Louise Fla­
mant, Josée Goffin et Solange Grand­
jean ; 6) Intermèdes et tombola ; 7) 
€ Vers l'Avenir », exécuté par la Fan-
lare.

3. La Fanfare sera dirigée par M.

4. Emile 
Binet, organiste de l'église Saint-Mar­
tin, à Arlon.

5. VAUX-LEZ-ROSIERES 
Succès théâtral 
Le dimanche de Pâques, la dramati­
que locale s'est taillée un magnifique 
succès en interprétant : t Gosse de 
misère », grand drame humain en 5 ac­
tes, de Mathilde Mortier, et « Notre 
nouveau Malt' d'école », la fine comédie 
d'A.

6. jeux de lumière très bien appro

Calcul de la polarité et subjectivité

In [8]:
results = []

for s in sample_sentences:
    blob = tb(s)
    sentiment = blob.sentiment
    results.append((s, sentiment[0], sentiment[1]))


df_sent = pd.DataFrame(results, columns=["phrase", "polarité", "subjectivité"])
df_sent

,phrase,polarité,subjectivité
0,Madeleine Van Mullem ; 4) « La \nVierge du Por...,0.100000,0.000000
1,"Cartier-Bresson « L'E-\n: toile », Interorétée...",0.000000,0.000000
2,La Fanfare sera dirigée par M.,0.000000,0.000000
3,"Emile \nBinet, organiste de l'église Saint-Mar...",0.000000,0.000000
4,VAUX-LEZ-ROSIERES \nSuccès théâtral \nLe diman...,0.226250,0.337500
5,jeux de lumière très bien appropriés.,0.000000,0.200000
6,"D'avance, on se \n1 réjouit déjà de la saison ...",0.150000,0.100000
7,"Quant à la comédie, disons tout de \nsuite qu'...",0.213333,0.133333
8,"Aussi, nous \nnous bornerons à signaler qu'ell...",0.000000,0.000000
9,Le même spectacle sera rendu ce di­\nmanche 16...,0.000000,0.500000


### Création d'une fonction `get_sentiment`

In [4]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity_perc}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")

### Analyser le sentiment d'une phrase

In [ ]:
get_sentiment("Ce journal est vraiment super intéressant.")

In [ ]:
get_sentiment("Cette phrase est négative et je ne suis pas content !")

## Utilisation de transformers

Importation du pipeline Transformers

In [ ]:
from transformers import pipeline

## 2. Utilisation de transformers

Documentation: https://github.com/TheophileBlard/french-sentiment-analysis-with-bert

**!!** Si le code ne tourne pas sur votre machine, vous pouvez le tester directement sur Google Colab en utilisant [ce lien](https://colab.research.google.com/github/TheophileBlard/french-sentiment-analysis-with-bert/blob/master/colab/french_sentiment_analysis_with_bert.ipynb) **!!**

Le modèle peut également être testé en ligne sur [HuggingFace](https://huggingface.co/tblard/tf-allocine)

### Installation des librairies et imports

In [ ]:
%pip install tensorflow
%pip install sentencepiece
%pip install transformers
%pip install tf_keras

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline

### Chargement du modèle

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine", use_pt=True)
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

sentiment_analyser = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

### Analyser le sentiment d'une phrase

In [39]:
sentiment_analyser("Ce journal est vraiment super intéressant.")

In [ ]:
sentiment_analyser("Cette phrase est négative et je ne suis pas content !")